In [1]:
!git clone https://github.com/giambono/divine_semantics.git
import os
os.chdir("/content/divine_semantics")
!pip install -r requirements.txt

Cloning into 'divine_semantics'...
remote: Enumerating objects: 207, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 207 (delta 35), reused 54 (delta 17), pack-reused 126 (from 1)
Receiving objects: 100% (207/207), 109.48 MiB | 16.83 MiB/s, done.
Resolving deltas: 100% (84/84), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
!pip install scikit-optimize

import pandas as pd
from src.find_similarity import find_most_similar_ensemble
from src.performance import evaluate_performance
from src.optimize_weights import optimize_weights

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 9.3 MB/s eta 0:00:00


In [9]:
from sentence_transformers import SentenceTransformer
df = pd.read_pickle("out/ensemble_all_embeddings.pkl")
# df = df[['volume', 'canto', 'verse', 'dante', 'singleton', 'musa', 'kirkpatrick', 'durling']]

In [10]:
test_queries = pd.read_parquet("data/paraphrased_verses.parquet")
test_queries = test_queries[["transformed_text", "expected_index"]]
test_queries = dict(zip(test_queries.iloc[:, 0], test_queries.iloc[:, 1]))

In [6]:
models={"multilingual_e5": SentenceTransformer("intfloat/multilingual-e5-large")}

In [12]:
best_weights = optimize_weights(df, ["singleton", "musa", "kirkpatrick", "durling"], models, test_queries)
best_weights

Streaming output truncated to the last 5000 lines.
  -0.00772926]
 ...
 [ 0.02439878 -0.00501365 -0.02966022 ... -0.01330578 -0.01824889
   0.01714332]
 [ 0.05765435 -0.00312472 -0.02114701 ... -0.01999305 -0.00432578
   0.01146715]
 [-0.00741977  0.00866382 -0.02683598 ... -0.00412435 -0.02167181
  -0.00853113]]
[[ 0.01610053 -0.02416307 -0.01733357 ... -0.00590286 -0.03375016
  -0.01746248]
 [ 0.03686344  0.00109443 -0.0245108  ... -0.02091643 -0.01358217
   0.0109821 ]
 [ 0.01740523 -0.01038312 -0.0378397  ... -0.01923795 -0.02833773
  -0.00391327]
 ...
 [ 0.02533809 -0.0082257  -0.0219347  ... -0.01098276 -0.02813654
   0.01912552]
 [ 0.06137166 -0.00751629 -0.01815173 ... -0.00979061  0.00270528
   0.01907368]
 [ 0.00235967  0.00799138 -0.01754851 ... -0.0132171  -0.02728047
  -0.00748969]]
[[ 0.01966798 -0.02943916 -0.01038687 ... -0.01155621 -0.02490427
  -0.00930514]
 [ 0.04734374 -0.00533824 -0.03602142 ... -0.01724925 -0.03015617
   0.00817549]
 [ 0.01232416 -0.00667303 -0.04

{'singleton': 0.015966252220214198,
 'musa': 0.23089382562214905,
 'kirkpatrick': 0.24102546602601177,
 'durling': 0.6832635188254583}

In [13]:
best_weights

{'singleton': 0.015966252220214198,
 'musa': 0.23089382562214905,
 'kirkpatrick': 0.24102546602601177,
 'durling': 0.6832635188254583}

In [ ]:
from src.compute import compute_ensemble_embeddings
models={"multilingual_e5": SentenceTransformer("intfloat/multilingual-e5-large")}

weights = {
    "dante": 0.0,
    "singleton": 0.1,
    "musa": 0.3,
    "kirkpatrick": 0.3,
    "durling": 0.3,
}

df = compute_ensemble_embeddings(df, columns=["singleton", "musa", "kirkpatrick", "durling"], models=models, weights=weights)
df.to_pickle(("out/ensemble_all_embeddings.pkl"))

Computing embeddings with multilingual_e5...


In [ ]:
test_queries = pd.read_pickle("out/test_set.pkl")
test_queries = test_queries[["query", "expected_index"]]
test_queries = dict(zip(test_queries.iloc[:, 0], test_queries.iloc[:, 1]))

In [ ]:
df, performance_results = evaluate_performance(df, models, test_queries)


Starting performance evaluation...
Evaluating multilingual_e5...


In [ ]:
performance_results

{'multilingual_e5': {'embedding_multilingual_e5_singleton': 0.845213849287169,
  'embedding_multilingual_e5_musa': 0.7871690427698574,
  'embedding_multilingual_e5_kirkpatrick': 0.8054989816700611,
  'embedding_multilingual_e5_durling': 0.8503054989816701,
  'embedding_multilingual_e5_avg': 0.8762729124236253,
  'embedding_multilingual_e5_weighted': 0.8828920570264766}}